In [1]:
import sys
import os
%load_ext autoreload
%autoreload 2

sys.path.append('../')
os.chdir('../')

In [2]:
import re

string = "res=[0, 1, 2, 5]"
pattern = r"res=\[(.*?)\]"
result = re.findall(pattern, string)[0]
lst = [int(num) for num in result.split(", ")]
print(lst)


[0, 1, 2, 5]


In [4]:
import glob
global_path = './outputs_permute'

all_result_paths = glob.glob(global_path + f'/**/result.csv', recursive=True)
all_result_paths

['./outputs_permute/mods_residual_exp/mods_sd240_T2405211220_res=[0, 3, 4, 5]_pmTrue/mods_sd240_T2405211220_res=[0, 3, 4, 5]_pmTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211230_res=[0, 1, 2, 4, 5]_pmTrue/mods_sd240_T2405211230_res=[0, 1, 2, 4, 5]_pmTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211228_res=[0, 1, 3, 4, 5]_pmTrue/mods_sd240_T2405211228_res=[0, 1, 3, 4, 5]_pmTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211214_res=[0, 4, 5]_pmTrue/mods_sd240_T2405211214_res=[0, 4, 5]_pmTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211216_res=[0, 1, 5]_pmTrue/mods_sd240_T2405211216_res=[0, 1, 5]_pmTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211227_res=[0, 2, 3, 4, 5]_pmTrue/mods_sd240_T2405211227_res=[0, 2, 3, 4, 5]_pmTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211219_res=[2, 3, 4, 5]_pmTrue/mods_sd240_T2405211219_res=[2, 3, 4, 5]

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re

critical_keys1 = ['']

save = True

global_path = './outputs'

for c1 in critical_keys1:
    all_result_paths = glob.glob(global_path + f'/residual_exp/**/result.csv', recursive=True) + glob.glob(global_path + f'/nope_residual_exp/**/result.csv', recursive=True)
    all_result_paths = [p for p in all_result_paths if '[' in p]

    result_summary = {}
    n_layers = 6
    for result_path in all_result_paths:
        result = pd.read_csv(result_path)
        top_test_result = result['test_acc'].max()
        pe_type = 'NoPE' if 'nope' in result_path else 'Original'
        pattern = r"res=\[(.*?)\]"
        residual_list = re.findall(pattern, result_path)[0]
        residual_list = [int(num) for num in residual_list.split(", ")] if residual_list!='' else []
        # residual_list = [int(num) for num in residual_list.split(", ")]
        missing_rc = [i for i in range(n_layers) if i not in residual_list]
        missing_rc_str = '{' + ','.join([str(i) for i in missing_rc])+ '}'


        if pe_type not in result_summary:
            result_summary[pe_type] = {}
        if missing_rc_str not in result_summary[pe_type]:
            result_summary[pe_type][missing_rc_str] = []
        result_summary[pe_type][missing_rc_str].append(top_test_result)
        if pe_type == 'Original' and missing_rc_str == '{5}':
            print(result_path, pe_type, missing_rc_str, top_test_result)

    from ast import literal_eval
    mean_results = {}
    for pe_type in result_summary:
        cur_type_dict = result_summary[pe_type].copy()
        for missing_rc in cur_type_dict:
            cur_type_dict[missing_rc] = len(cur_type_dict[missing_rc])
        mean_results[pe_type] = cur_type_dict

    df = pd.DataFrame(mean_results)
    missing0 = ['{}']
    missing1 = [str({i}) for i in range(n_layers)]
    missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
    missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
    missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
    missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
    from IPython.display import display
    display(df.loc[missing0,:].T)
    display(df.loc[missing1,:].T)
    display(df.loc[missing2,:].T)
    display(df.loc[missing3,:].T)

    num_seeds = df.values.min()


    from collections import OrderedDict
    final_results = OrderedDict()
    for pe_type in {'Original', 'NoPE'}:
        cur_type_dict = result_summary[pe_type].copy()
        final_dict = {
            'min': cur_type_dict.copy(),
            'max': cur_type_dict.copy(),
            'avg.': cur_type_dict.copy(),
        }
        for missing_rc in cur_type_dict:
            final_dict['avg.'][missing_rc] = np.mean(cur_type_dict[missing_rc])
            final_dict['min'][missing_rc] = np.min(cur_type_dict[missing_rc])
            final_dict['max'][missing_rc] = np.max(cur_type_dict[missing_rc])

        for k in final_dict:
            final_results[pe_type+' ('+k+')'] = final_dict[k]

    df = pd.DataFrame(final_results)
    missing1 = [str({i}) for i in range(n_layers)]
    missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
    missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
    missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
    missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
    from IPython.display import display
    r1 = df.loc[missing1,:].T
    r2 = df.loc[missing2,:].T
    r3 = df.loc[missing3,:].T
    
    r0 = df.loc[['{}'],:].T
    display(r0)
    display(r1)
    display(r2)
    display(r3)

    df = pd.DataFrame(final_results)
    missing1 = [str({i}) for i in range(n_layers)]
    missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
    missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
    missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
    missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
    from IPython.display import display
    r1 = df.loc[missing1,:].T
    r2 = df.loc[missing2,:].T
    r3 = df.loc[missing3,:].T
    # r4 = df.loc[missing4,:].T
    # r5 = df.loc[missing5,:].T
    # r6 = df.loc[missing6,:].T
    r0 = df.loc[['{}'],:].T
    display(r0)
    display(r1)
    display(r2)
    display(r3)
    # display(r4)
    # display(r5)

    # write all 3 to the same csv file
    if save:
        all_r = pd.concat([r0, r1, r2, r3,], axis=1)
        
        all_r.to_csv(f'{c1}_{num_seeds}seeds.csv')



./outputs/residual_exp/addition_reverse_sd111_T2402031302_res=[0, 1, 2, 3, 4]/addition_reverse_sd111_T2402031302_res=[0, 1, 2, 3, 4]/result.csv Original {5} 99.94949494949496
./outputs/residual_exp/addition_reverse_sd222_T2402012039_res=[0, 1, 2, 3, 4]/addition_reverse_sd222_T2402012039_res=[0, 1, 2, 3, 4]/result.csv Original {5} 99.86868686868688
./outputs/residual_exp/addition_reverse_sd555_T2402030512_res=[0, 1, 2, 3, 4]/addition_reverse_sd555_T2402030512_res=[0, 1, 2, 3, 4]/result.csv Original {5} 99.91919191919192
./outputs/residual_exp/addition_reverse_sd666_T2402030643_res=[0, 1, 2, 3, 4]/addition_reverse_sd666_T2402030643_res=[0, 1, 2, 3, 4]/result.csv Original {5} 99.90909090909092
./outputs/residual_exp/addition_reverse_sd333_T2402012201_res=[0, 1, 2, 3, 4]/addition_reverse_sd333_T2402012201_res=[0, 1, 2, 3, 4]/result.csv Original {5} 99.87878787878788
./outputs/residual_exp/addition_reverse_sd444_T2402030302_res=[0, 1, 2, 3, 4]/addition_reverse_sd444_T2402030302_res=[0, 1, 2

,{}
Original,6
NoPE,6


,{0},{1},{2},{3},{4},{5}
Original,6,6,6,6,6,6
NoPE,6,6,6,6,6,6


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
Original,7,7,7,7,7
NoPE,7,7,7,7,7


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
Original,8,8,8,8
NoPE,8,8,8,8


...

,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
Original (min),0.010101,0.010101,0.010101,0.010101
Original (max),0.818182,0.020202,0.030303,0.020202
Original (avg.),0.401515,0.013889,0.016414,0.016414
NoPE (min),0.020202,0.010101,0.010101,0.010101
NoPE (max),0.131313,0.090909,0.030303,0.040404
NoPE (avg.),0.082071,0.025253,0.016414,0.020202


,{}
Original (min),100.000000
Original (max),100.000000
Original (avg.),100.000000
NoPE (min),99.292929
NoPE (max),99.757576
NoPE (avg.),99.587542


,{0},{1},{2},{3},{4},{5}
Original (min),99.919192,99.707071,99.343434,99.030303,99.575758,99.858586
Original (max),100.000000,99.949495,99.666667,99.989899,99.989899,99.949495
Original (avg.),99.973064,99.836700,99.508418,99.754209,99.856902,99.897306
NoPE (min),96.535354,91.272727,83.303030,50.464646,89.858586,98.818182
NoPE (max),97.636364,98.484848,96.868687,87.878788,98.979798,99.727273
NoPE (avg.),96.964646,95.456229,89.826599,69.132997,95.993266,99.476431


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
Original (min),98.525253,0.010101,0.010101,0.010101,0.010101
Original (max),99.737374,90.747475,2.515152,99.979798,99.616162
Original (avg.),99.194805,15.414141,0.392496,28.598846,29.148629
NoPE (min),10.363636,0.010101,0.010101,0.010101,0.010101
NoPE (max),80.121212,0.151515,0.070707,0.070707,0.686869
NoPE (avg.),53.522367,0.069264,0.046176,0.033189,0.160173


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
Original (min),0.010101,0.010101,0.010101,0.010101
Original (max),0.818182,0.020202,0.030303,0.020202
Original (avg.),0.401515,0.013889,0.016414,0.016414
NoPE (min),0.020202,0.010101,0.010101,0.010101
NoPE (max),0.131313,0.090909,0.030303,0.040404
NoPE (avg.),0.082071,0.025253,0.016414,0.020202


In [8]:
f'{c1}_{num_seeds}seeds.csv'

'_1seeds.csv'